<a href="https://colab.research.google.com/github/sunshineluyao/CV_XAI/blob/master/AutomatedTradingStrategy/ma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class MA:
  def __init__(self, short_window=30, long_window=180,Start_date='2013-12-27',initial_capital=100000,max_buy =1000000, max_sell=1000000, gas = 0.001, URL ='https://raw.githubusercontent.com/sunshineluyao/CVML2021/main/BTC_2020_12_31.csv'):
    self.short = short_window
    self.long = long_window
    self.start = Start_date
    self.initial_capital = initial_capital
    self.max_buy = max_buy
    self.max_sell = max_sell
    self.gas = gas
    self.URL = URL
  def portfolio(self):
    import pandas as pd 
    import numpy as np
    pd.options.display.float_format = "{:,.2f}".format
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    data=pd.read_csv(self.URL,index_col='Unnamed: 0')
    data.index=pd.to_datetime(data.index)
    data = data[data.index>=self.start]
    prices = data['PriceUSD']
    data['short_ma'] = data['PriceUSD'].rolling(window=self.short, min_periods=1, center=False).mean()
    data['long_ma'] = data['PriceUSD'].rolling(window=self.long, min_periods=1, center=False).mean()
    data['position'] = np.where(data['short_ma'] > data['long_ma'], 1.0, 0.0) 
    data['signal'] = data['position'].diff() 
    data=data.fillna(0)
    signals = data['signal']
    cashes = []
    bitcoins = []
    holdings = []
    states = []
    cash = self.initial_capital
    bitcoin = 0
    holding = 0
    signal = ""
    state = ""
    def buy(i, cash, bitcoin, price):
      units = cash/(price*(1+self.gas))
      if units < 1:
        buy_units = 0
      else:
        if units>self.max_buy:
          buy_units = self.max_buy
        else:
          buy_units = units
        print('index %d: buy %f units at price %.3f, current cash %.3f, current bitcoin %.3f' % (i, buy_units, price, cash, bitcoin))
      cost = buy_units*price*(1+self.gas)
      cash -= cost
      bitcoin += buy_units
      holding = bitcoin*price
      return buy_units, cash, bitcoin, holding
    def sell(i, cash, bitcoin, price):
      if bitcoin == 0:
        sell_units = 0
      else:
        if bitcoin > self.max_sell:
          sell_units = self.max_sell
        else:
          sell_units = bitcoin
        print('index %d: sell %.3f units at price %.3f, current cash %.3f, current bitcoin %.3f' % (i, sell_units, price, cash, bitcoin))
      bitcoin -=sell_units
      revenue = sell_units*price*(1-self.gas)
      cash += revenue
      holding = bitcoin*price
      return sell_units, cash, bitcoin, holding
    for i in range(0,data.shape[0]):
      price = prices[i]
      signal = signals[i]
      if signal == 1:
        buy_units, cash, bitcoin, holding = buy(i, cash, bitcoin, price)
        if buy_units>0:
          state = 1
        else:
          state = 0  
      elif signal == -1:
        sell_units, cash, bitcoin, holding = sell(i, cash, bitcoin, price)
        if sell_units>0:
          state = -1
        else:
          state = 0
      else:
        state = 0
        bitcoin = bitcoin
        cash = cash
        holding =bitcoin*price ### holding updates when prices update even when there is no buy or sell actions
      states.append(state)
      cashes.append(cash)
      bitcoins.append(bitcoin)
      holdings.append(holding)

    data.loc[:,'state']=states
    data.loc[:,'cash']=cashes
    data.loc[:,'bitcoin']=bitcoins
    data.loc[:,'holding']=holdings
    data['total']=data['cash']+data['holding']
    data['roi_gross'] = (data['total']-self.initial_capital)/self.initial_capital*100
    data['roi_daily'] = data['total'].pct_change()
    data=data.fillna(0)
    data['sharpe'] = 0
    for i in range(0, data.shape[0]):
      if data['roi_daily'].iloc[0:i].std()==0:
        data['sharpe'].iloc[i]=0
      else:
        data['sharpe'].iloc[i]=(i**0.5)*np.mean(data['roi_daily'].iloc[0:i])/data['roi_daily'].iloc[0:i].std()
    data=data.fillna(0)
    return data
  def figure_signal(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name =str(self.short)+'-day Moving Average',
            x=data.index,
            y=data['short_ma'],
            line_color="pink",
            line_width=1,

        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name = str(self.long)+'-day Moving Average',
            x=data.index,
            y=data['long_ma'],
            line_color="blue",
            line_width=1,

        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Price in USD',
            x=data.index,
            y=data['PriceUSD'],
            line_color="black",
            line_width=1
        ),
        secondary_y=False,
    )
    
    fig.add_trace(
        go.Scatter(name="buy",mode="markers", x=data.index[data.state==1], y=data['PriceUSD'][data.state==1], marker_symbol=119,
                              marker_line_color="green", marker_color="green",
                              marker_line_width=2, marker_size=15,
    ),
    secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(name="sell",mode="markers", x=data.index[data.state==-1], y=data['PriceUSD'][data.state==-1], marker_symbol=120,
                              marker_line_color="red", marker_color="red",
                              marker_line_width=2, marker_size=15,
    ),
        secondary_y=False,
    )

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Price Moving Averages in USD</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Price in USD</b>", secondary_y=True)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Buy and Sell Signals: MA Crossover Rule"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return
  def figure_portfolio(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name ='Cash',
            x=data.index,
            y=data['cash'],
            line_color="red",
            line_width=1,

        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Holding',
            x=data.index,
            y=data['holding'],
            line_color="green",
            line_width=1
        ),
        secondary_y=False,
    )
    fig.add_trace(
        
        go.Scatter(
            mode='lines',
            name='Total',
            x=data.index,
            y=data['total'],
            line_color="blue",
            line_width=1
        ),
        secondary_y=False,
    )
    # Set y-axes titles
    fig.update_yaxes(title_text="<b>USD</b>", secondary_y=False)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Portfolio Time Series: MA Crossover Automated Trading Strategies"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return 
  def figure_returns(self,data):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
          
        go.Scatter(
              mode='lines',
              name ='Gross ROI',
              x=data.index,
              y=data['roi_gross'],
              line_color="blue",
              line_width=1,

          ),
        secondary_y=False,
      )
    fig.add_trace(
          
        go.Scatter(
              mode='lines',
              name='Annualized Sharpe Ratio',
              x=data.index,
              y=data['sharpe'],
              line_color="black",
              line_width=1
          ),
        secondary_y=True,
      )
    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Percent</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Ratio</b>", secondary_y=True)
    # Set y-axes title
    fig.update_xaxes(title_text="<b>date<b>")
    # set title 
    fig.update_layout(
        title_text="Gross ROI and Annualized Sharpe Ratio: MA Crossover Strategy"
    )
    # update templates
    fig.update_layout(template="plotly_white")
    fig.show()
    return
